In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from torchvision.models import resnet50, ResNet50_Weights
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy



# Model

In [3]:
def conv_block(in_f, out_f, activation='relu', *args, **kwargs):
    activations = nn.ModuleDict([
                ['lrelu', nn.LeakyReLU()],
                ['relu', nn.ReLU()]
    ])
    
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.MaxPool2d(2, 2),
        activations[activation]
    )

class Encoder(nn.Module):
    def __init__(self,encoder_sizes,*args,**kwargs):
        super().__init__()
        self.encoced_blocks = nn.Sequential(*[conv_block(in_f, out_f, kernel_size=3, padding=1, *args, **kwargs) 
                       for in_f, out_f in zip(encoder_sizes, encoder_sizes[1:])])
        
    def forward(self, x):
        return self.encoced_blocks(x)

class Predictor(nn.Module):
    def __init__(self,num_classes,connector_size):
        super().__init__();
        self.predictor=nn.Sequential(nn.Linear(connector_size * 4 * 4, 256),nn.ReLU(),nn.Dropout(0.5),nn.Linear(256, num_classes))

    def forward(self, x):
        return self.predictor(x)
        
class CNN(nn.Module):
    def __init__(self, in_c,enc_sizes, num_classes=10,activation='relu'):
        super().__init__()

        self.enc_sizes = [in_c, *enc_sizes]

        self.encoder = Encoder(self.enc_sizes, activation=activation)
        self.predictor=Predictor(num_classes,self.enc_sizes[-1])

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.predictor(x);
        return x


In [4]:
def conv_block(in_f, out_f, activation='relu', *args, **kwargs):
    activations = nn.ModuleDict([
                ['lrelu', nn.LeakyReLU()],
                ['relu', nn.ReLU()]
    ])
    
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.MaxPool2d(2, 2),
        activations[activation]
    )

class Encoder(nn.Module):
    def __init__(self,encoder_sizes,*args,**kwargs):
        super().__init__()
        self.encoced_blocks = nn.Sequential(*[conv_block(in_f, out_f, kernel_size=3, padding=1, *args, **kwargs) 
                       for in_f, out_f in zip(encoder_sizes, encoder_sizes[1:])])
        
    def forward(self, x):
        return self.encoced_blocks(x)

class Predictor2(nn.Module):
    def __init__(self,num_classes,connector_size):
        super().__init__();
        self.predictor=nn.Sequential(nn.Linear(connector_size * 4 * 4, num_classes))

    def forward(self, x):
        return self.predictor(x)
        
class CNN2(nn.Module):
    def __init__(self, in_c,enc_sizes, num_classes=10,activation='relu'):
        super().__init__()

        self.enc_sizes = [in_c, *enc_sizes]

        self.encoder = Encoder(self.enc_sizes, activation=activation)
        self.predictor=Predictor2(num_classes,self.enc_sizes[-1])

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.predictor(x);
        return x


# Train Test Utils

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy
from sklearn.metrics import precision_recall_fscore_support

def train_model_extra_metric(model, train_loader, val_loader, num_epochs=10, lr=0.001,lambda_reg=0.01, device='cpu', early_stopping_patience=5):
    """
    Trains a PyTorch model with validation and calculates precision, recall, and F1-score.
    
    Args:
        model (nn.Module): The PyTorch model to train.
        train_loader (DataLoader): The training data loader.
        val_loader (DataLoader): The validation data loader.
        num_epochs (int, optional): The number of epochs to train for. Defaults to 10.
        lr (float, optional): The learning rate. Defaults to 0.001.
        device (str, optional): The device to use for training ('cpu' or 'cuda'). Defaults to 'cpu'.
        early_stopping_patience (int, optional): Number of epochs to wait before early stopping. Defaults to 5.
    
    Returns:
        tuple: (best_model, dict_metrics) where dict_metrics contains training history
    """
    # Initialize model and optimization
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # Initialize tracking variables
    best_val_f1 = 0
    best_val=0
    best_model = None
    patience_counter = 0
    history = {
        'train_loss': [],
        'val_accuracy': [],
        'val_precision': [],
        'val_recall': [],
        'val_f1': []
    }
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        num_batches = 0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            # l2_norm = sum(p.pow(2).sum() for p in model.parameters())
            # loss += lambda_reg * l2_norm
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1
        
        avg_train_loss = total_loss / num_batches
        history['train_loss'].append(avg_train_loss)
        
        # Validation phase
        if val_loader:
            model.eval()
            correct = 0
            total = 0
            all_targets = []
            all_predictions = []
            
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    predictions = outputs.argmax(dim=1)
                    
                    correct += (predictions == targets).sum().item()
                    total += targets.size(0)
                    
                    all_targets.extend(targets.cpu().numpy())
                    all_predictions.extend(predictions.cpu().numpy())
            
            # Calculate metrics
            val_accuracy = correct / total
            precision, recall, f1, _ = precision_recall_fscore_support(
                all_targets, 
                all_predictions, 
                average='weighted', 
                zero_division=0
            )
            
            # Update history
            history['val_accuracy'].append(val_accuracy)
            history['val_precision'].append(precision)
            history['val_recall'].append(recall)
            history['val_f1'].append(f1)
            
            # Print progress
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train Loss: {avg_train_loss:.4f}")
            print(f"Val Accuracy: {val_accuracy:.4f}")
            print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
            print("-" * 50)
            
            # Model checkpoint and early stopping
            if val_accuracy>best_val:
                best_model=copy.deepcopy(model)
                best_val=val_accuracy
                patience_counter = 0
            else:
                patience_counter += 1
            
            # Early stopping check
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping triggered after epoch {epoch+1}")
                break
    
    return best_model

In [6]:
def train_model(model, train_loader, val_loader, num_epochs=10, lr=0.001, device='cpu'):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    best_val=0;
    best_model=model;
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            # l2_norm = sum(p.pow(2).sum() for p in model.parameters())
            # loss += 0.01 * l2_norm
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        
        # Validation (optional in minimal version)
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                predictions = outputs.argmax(dim=1)
                correct += (predictions == targets).sum().item()
                total += targets.size(0)
        val_accuracy = correct / total
        if val_accuracy>best_val:
            best_model=copy.deepcopy(model)
            best_val=val_accuracy
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    return best_model;


In [7]:
def test_model(model,test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predictions = outputs.argmax(dim=1)
            correct += (predictions == targets).sum().item()
            total += targets.size(0)
    test_accuracy = correct / total
    print('test accuracy',test_accuracy)
    return test_accuracy

In [8]:
def remove_keys(original_dict, keys_to_remove):
    # Create a new dictionary by excluding the specified keys
    return {key: value for key, value in original_dict.items() if key not in keys_to_remove}

# Other metrics

In [9]:
from collections import OrderedDict

def state_dict_to_vector(state_dict, remove_keys=[]):

    shared_state_dict = copy.deepcopy(state_dict)

    for key in remove_keys:

        if key in shared_state_dict:

            del shared_state_dict[key]

    sorted_shared_state_dict = OrderedDict(sorted(shared_state_dict.items()))

    return torch.nn.utils.parameters_to_vector([value.reshape(-1) for key, value in sorted_shared_state_dict.items()])





def vector_to_state_dict(vector, state_dict, remove_keys=[]):

    # create a reference dict to define the order of the vector

    reference_dict = copy.deepcopy(state_dict)

    for key in remove_keys:

        if key in reference_dict:

            del reference_dict[key]

    sorted_reference_dict = OrderedDict(sorted(reference_dict.items()))



    # create a shared state dict using the refence dict

    torch.nn.utils.vector_to_parameters(vector, sorted_reference_dict.values())



    # add back the encoder and decoder embedding weights.

    if "transformer.shared.weight" in sorted_reference_dict:

        for key in remove_keys:

            sorted_reference_dict[key] = sorted_reference_dict["transformer.shared.weight"]

    return sorted_reference_dict

In [10]:
import torch

def count_sign_conflicts(tensor: torch.Tensor) -> int:
    # Get the sign of each element in the tensor
    # -1 for negative, 1 for positive, and 0 for zero
    signs = torch.sign(tensor).to(device)

    # Create a mask for non-zero entries
    non_zero_mask = (signs != 0)

    # Check for sign conflicts, ignoring zeros
    negative_mask = (signs == -1) & non_zero_mask
    positive_mask = (signs == 1) & non_zero_mask

    # A conflict occurs if a column contains both -1 and 1
    conflict_mask = negative_mask.any(dim=0) & positive_mask.any(dim=0)

    # Count the number of columns with sign conflicts
    num_conflicts = conflict_mask.sum().item()

    return num_conflicts

def calc_sign_conflicts(model_list,dummy_model,exclude_keys,model_class,number_of_tasks, is_transformer=False):
    device='cuda' if torch.cuda.is_available() else 'cpu'

    # dummy_model=model_class(3,enc_sizes,num_classes=2).to(device)  
    dm_new=remove_keys(dummy_model.state_dict(),exclude_keys)
    flat_ptm= state_dict_to_vector(dm_new)

    flat_ft = torch.vstack([state_dict_to_vector(remove_keys(model_list[x].state_dict(),exclude_keys) )for x in range(number_of_tasks)]);
  
    tv_flat_checks = flat_ft - flat_ptm;
    # torch.save(tv_flat_checks,model_name+"_"+task_name+"_"+str(number_of_tasks)+"_"+version+".pt");
    # plot_mds_torch(tv_flat_checks);
    # plot_tsne_torch(tv_flat_checks)
    # flipped=flip(tv_flat_checks)
    # plot_mds_torch(flipped)
    # plot_tsne_torch(flipped)
    return count_sign_conflicts(tv_flat_checks)/tv_flat_checks.shape[1], tv_flat_checks.shape[1];


# Confusion Matrix

In [11]:
def compute_confusion_matrix(model, data_loader, num_classes, device='cpu', class_names=None, plot=False):
    """
    Compute and optionally plot the confusion matrix for a model.

    Args:
        model: Trained PyTorch model.
        data_loader: DataLoader for the dataset to evaluate.
        num_classes: Number of classes in the dataset.
        device: Device to use ('cpu' or 'cuda').
        class_names: List of class names (optional, used for plotting).
        plot: Boolean, whether to plot the confusion matrix.

    Returns:
        confusion_matrix: Normalized confusion matrix as a NumPy array.
    """
    model.eval()
    confusion_matrix = np.zeros((num_classes, num_classes))
    
    # Compute the confusion matrix
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predictions = outputs.argmax(dim=1).cpu().numpy()
            targets = targets.cpu().numpy()
            for t, p in zip(targets, predictions):
                confusion_matrix[t, p] += 1
    
    # Normalize rows to sum to 1
    row_sums = confusion_matrix.sum(axis=1, keepdims=True)
    confusion_matrix = confusion_matrix / (row_sums + 1e-7)  # Avoid division by zero

    # Optionally plot the confusion matrix
    if plot:
        plt.figure(figsize=(10, 8))
        sns.heatmap(confusion_matrix, annot=True, fmt=".2f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
        plt.title("Normalized Confusion Matrix")
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.show()
    
    return confusion_matrix

In [12]:
import logging
import numpy as np

__author__ = 'Otilia Stretcu'


def _find_closest_cluster_min(dist_matrix, clusters, node_to_cluster):
    n = dist_matrix.shape[0]
    closest_cluster = []
    closest_cluster_dists = []
    # For each cluster, find the edge of min weight going out of it.
    for cluster in clusters:
        cluster_nodes = np.asarray(cluster)
        # Create a mask that marks the nodes in the current cluster.
        cluster_mask = np.zeros((n,), dtype=np.bool)
        cluster_mask[cluster_nodes] = True
        non_cluster_mask = np.logical_not(cluster_mask)
        non_cluster_nodes = np.where(non_cluster_mask)[0]
        # Only consider the edges from nodes belonging to the current
        # cluster and nodes from a different cluster.
        dists = dist_matrix[cluster_mask][:, non_cluster_mask]
        # Find the min of these distances and the node from a different
        # cluster this connects to.
        if len(dists.shape) < 2:
            if len(cluster_nodes) == 1:
                dists = dists[None]
            else:
                dists = dists[:, None]
        min_dists = np.amin(dists, axis=1)
        cluster_node_id = np.argmin(min_dists)
        non_cluster_node_id = np.argmin(dists[cluster_node_id])
        smallest_dist = dists[cluster_node_id][non_cluster_node_id]
        non_cluster_node_id = non_cluster_nodes[non_cluster_node_id]
        # Find the cluster of this closest node.
        closest_cluster.append(node_to_cluster[non_cluster_node_id])
        closest_cluster_dists.append(smallest_dist)

    return closest_cluster, closest_cluster_dists


def find_closest_cluster(dist_matrix, clusters):
    """Finds the closest cluster to each cluster.

    Arguments:
        dist_matrix: A numpy array representing a distance matrix between all pairs of nodes.
        clusters: A list of lists, where each inner list contains the node ids that are in that
            cluster.
    Returns:
        closest_cluster
        closest_cluster_dists
    """
    # Compute the number of nodes.
    n = dist_matrix.shape[0]
    # Map each node id to its cluster.
    node_to_cluster = np.zeros((n,), dtype=np.int)
    for cluster_id, cluster in enumerate(clusters):
        for node in cluster:
            node_to_cluster[node] = cluster_id

    # For each cluster, find the nearest cluster to it, where nearest is
    # defined depending on the linkage method.
    closest_cluster, closest_cluster_dists = _find_closest_cluster_min(
        dist_matrix, clusters, node_to_cluster)

    return closest_cluster, closest_cluster_dists


def merge_clusters(clusters, closest_cluster, num_clusters, last_cluster_id):
    # Merge each cluster with the one closest to it. This will lead to a
    # list of cluster sets.
    color = ['not_visited' for _ in range(num_clusters)]
    cluster_to_set = np.zeros((num_clusters,), dtype=np.int)
    cluster_sets = []
    cluster_set_to_id = []
    for i in range(num_clusters):
        if color[i] == 'not_visited':
            current_set = {i}
            color[i] = 'in_progress'
            j = i
            while closest_cluster[j] is not None and color[closest_cluster[j]] == 'not_visited':
                current_set.add(closest_cluster[j])
                color[closest_cluster[j]] = 'in_progress'
                j = closest_cluster[j]
            # If the cluster where we stopped doesn't have a closest cluster or it was visited
            # during the current traversal starting at i, then we form a new set.
            if closest_cluster[j] is None or color[closest_cluster[j]] == 'in_progress':
                # This a new set of clusters.
                set_index = len(cluster_sets)
                cluster_sets.append(current_set)
                last_cluster_id += 1
                cluster_set_to_id.append(last_cluster_id)
            else:
                # This has to be added to an existing set of clusters.
                set_index = cluster_to_set[closest_cluster[j]]
                cluster_sets[set_index].update(current_set)

            for j in list(current_set):
                color[j] = 'visited'
                cluster_to_set[j] = set_index
    # Expand the cluster sets to actual clusters.)
    clusters = [
        [elem for cluster_index in list(c)
         for elem in clusters[cluster_index]]
        for c in cluster_sets]
    return clusters


def affinity_clustering(dist_matrix, eps=1e-7):
    """Performs affinity clustering based on the provided distance matrix.

    Our implementation is based on a publication of Bateni et al., "Affinity clustering:
    Hierarchical clustering at scale." NeurIPS, 2017. Specifically, it is based on the authors'
    description of the algorithm, not on their released map-reduce-based code.

    Args:
        dist_matrix: A numpy array of shape (num_classes, num_classes) representing the distance
            matrix between labels.
        eps: A small float value representing the standard deviation of Gaussian noise added to
            all distances to break ties.
    Returns:
        A list of lists containing the label clusters per level. Each outer list corresponds to a
        level in the hierarchy. For each level, each inner list contains the label ids that are
        clustered together at this level. Levels are indexed from bottom to top.
        E.g., for the following hierarchy containing k=5 labels:
                           c
                        /     \
                       a      b
                     / | \   /\
                    1  2 3  4 5
        the returned list of lists will be:
        [[1, 2, 3, 4, 5], [[1, 2, 3], [4, 5]], [[1], [2], [3], [4], [5]].
    """
    # Add a small value eps to all edges to break ties.
    num_elem = dist_matrix.shape[0]
    dist_matrix = dist_matrix + np.random.rand(num_elem, num_elem) * eps

    # Num samples.
    n = dist_matrix.shape[0]
    # Each sample is it's own cluster in the beginning.
    clusters = [[i] for i in range(n)]
    # Keep track of clusters formed as we go.
    last_cluster_id = len(clusters) - 1

    # Merge clusters iteratively.
    num_clusters = len(clusters)
    clusters_per_level = [clusters]
    level = 0
    while num_clusters > 1:
        # For each cluster, find the cluster that is nearest to it.
        closest_cluster, closest_cluster_dists = find_closest_cluster(dist_matrix, clusters)

        # Merge each cluster with the one closest to it.
        new_clusters = merge_clusters(clusters, closest_cluster, num_clusters, last_cluster_id)

        # If everything was merged in the first iteration, we remove the most expensive edge.
        if level == 0:
            while len(new_clusters) == 1:
                logging.info('All nodes were merged in the first iteration. Removing the most '
                             'expensive edge and trying again.')
                idx = np.argmax(closest_cluster_dists)
                closest_cluster[idx] = None
                closest_cluster_dists[idx] = -np.inf
                new_clusters = merge_clusters(clusters, closest_cluster, num_clusters,
                                              last_cluster_id)

        num_clusters = len(new_clusters)
        clusters = new_clusters
        clusters_per_level.append(clusters)
        level += 1

    return clusters_per_level

# Load Dataset

In [13]:
# 1. Load CIFAR-10 Dataset
def load_cifar10(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))  # Normalize with CIFAR-10 stats
    ])
    
    train_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform
    )
    test_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform
    )
    
    # Split train dataset into training and validation
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
    
    # DataLoaders
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, len(train_dataset.classes),train_dataset.classes



In [14]:


# 1. Load CIFAR-10 Dataset
def load_cifar100(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))  # Normalize with CIFAR-10 stats
    ])
    
    train_dataset = torchvision.datasets.CIFAR100(
        root='./data', train=True, download=True, transform=transform
    )
    test_dataset = torchvision.datasets.CIFAR100(
        root='./data', train=False, download=True, transform=transform
    )
    
    # Split train dataset into training and validation
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
    
    # DataLoaders
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, len(train_dataset.classes),train_dataset.classes



In [15]:
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

# 1. Load MNIST Dataset
def load_mnist(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize(32),  # Resize images to 32x32 to match CIFAR-10 size
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # Normalize with MNIST stats (mean, std)
    ])
    
    train_dataset = torchvision.datasets.MNIST(
        root='./data', train=True, download=True, transform=transform
    )
    test_dataset = torchvision.datasets.MNIST(
        root='./data', train=False, download=True, transform=transform
    )
    
    # Split train dataset into training and validation
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
    
    # DataLoaders
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, len(train_dataset.classes), train_dataset.classes


In [16]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
import os
from sklearn.model_selection import train_test_split

def load_folder_dataset(data_path, valid=False, batch_size=64, test_size=0.1, val_size=0.1):
    """
    Load and prepare the dataset with train/validation/test split.
    
    Args:
        batch_size (int): Size of batches for DataLoader
        test_size (float): Fraction of data for testing
        val_size (float): Fraction of remaining data for validation after test split
        
    Returns:
        train_loader: DataLoader for training set
        val_loader: DataLoader for validation set
        test_loader: DataLoader for test set
        total_classes (int): Number of classes in the dataset
        classnames (list): List of class names
    """
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    ])
    
    if valid == False:
        # Load the full dataset
        full_dataset = datasets.ImageFolder(root=data_path, transform=transform)
        
        # Get class names and total number of classes
        classnames = full_dataset.classes
        total_classes = len(classnames)
        
        # Get indices for splitting the dataset
        indices = list(range(len(full_dataset)))
        
        # Split into training + (validation + test)
        train_indices, test_val_indices = train_test_split(
            indices,
            test_size=test_size,  # Test size, e.g., 10%
            stratify=[full_dataset.targets[i] for i in indices],
            random_state=42
        )
        
        # Split the remaining data (validation + test)
        val_indices, test_indices = train_test_split(
            test_val_indices,
            test_size=val_size / (test_size + val_size),  # Validation size within remaining data
            stratify=[full_dataset.targets[i] for i in test_val_indices],
            random_state=42
        )
        
        # Create datasets for train, validation, and test
        train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
        val_dataset = torch.utils.data.Subset(full_dataset, val_indices)
        test_dataset = torch.utils.data.Subset(full_dataset, test_indices)
    
    else:
        # Separate train, validation, and test folders
        train_dataset = datasets.ImageFolder(root=os.path.join(data_path, 'train'), transform=transform)
        val_dataset = datasets.ImageFolder(root=os.path.join(data_path, 'val'), transform=transform)
        test_dataset = datasets.ImageFolder(root=os.path.join(data_path, 'test'), transform=transform)
        classnames = train_dataset.classes
        total_classes = len(classnames)
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )
    
    return train_loader, val_loader, test_loader, total_classes, classnames


# Cluster

In [17]:
def compute_hierarchy_scipy(dist_matrix,symm=True, method='average', t=1.0, criterion='distance',depth=2):
    """
    Compute hierarchy using scipy's hierarchical clustering.

    Args:
        dist_matrix: A numpy array of shape (num_classes, num_classes) representing the distance matrix.
        method: Linkage method to use ('single', 'complete', 'average', etc.).
        t: Threshold for forming clusters.
        criterion: Criterion to use to form clusters ('distance', 'maxclust', etc.).
    
    Returns:
        clusters_per_level: A list of lists containing the label clusters per level.
    """
    # Step 1: Compute linkage matrix from the distance matrix
    # Convert distance matrix to condensed form (needed for scipy linkage)
    if dist_matrix is not None:
        if symm:
            dist_matrix = dist_matrix + dist_matrix.T
    condensed_dist_matrix = np.triu(dist_matrix, k=1)[np.triu_indices(dist_matrix.shape[0], k=1)]
    linkage_matrix = linkage(condensed_dist_matrix, method=method)
    # print(linkage_matrix)
    # Step 2: Determine clusters at different thresholds
    clusters_per_level = []
    num_classes = dist_matrix.shape[0]
    
    # Iterate over levels
    for threshold in np.linspace(0, t, num_classes):
        cluster_labels = fcluster(linkage_matrix, t=threshold, criterion=criterion)
        # Group labels into clusters
        clusters = [[] for _ in range(max(cluster_labels))]
        for label_idx, cluster_idx in enumerate(cluster_labels):
            clusters[cluster_idx - 1].append(label_idx)
        clusters_per_level.append(clusters)

    return clusters_per_level


In [18]:
from torch.utils.data import Dataset

class UpdatedDataset(Dataset):
    def __init__(self, original_dataset, new_labels):
        """
        Create a new dataset with updated labels.
        
        Args:
            original_dataset: The original dataset (e.g., train_loader.dataset).
            new_labels: The updated labels for the dataset.
        """
        self.original_dataset = original_dataset
        self.new_labels = new_labels

    def __getitem__(self, index):
        data, _ = self.original_dataset[index]  # Get the input data, ignore the original label
        new_label = self.new_labels[index]  # Get the updated label
        return data, new_label

    def __len__(self):
        return len(self.original_dataset)


def update_data_labels(original_dataset, new_labels, num_classes):
    """
    Update the labels of the dataset to reflect the new clusters at the current level.
    
    Args:
        original_dataset: The original dataset (e.g., train_loader.dataset).
        new_labels: A list or array of updated labels corresponding to the new clusters.
        num_classes: The number of clusters (classes) at this level of the hierarchy.
    
    Returns:
        A new dataset with updated labels.
    """
    if len(new_labels) != len(original_dataset):
        raise ValueError("The length of new_labels must match the size of the original dataset.")
    
    return UpdatedDataset(original_dataset, new_labels)

In [19]:
torch.manual_seed(33)
device='cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
conflict_folder="/kaggle/input/taskvector-distances"
conflicts_files=os.listdir(conflict_folder);
hyperParam={'ci':7,'dis':250,'dis_layer':70,'sim':4,'diseuc':40,'diseuc_layer':10, 'disci':40,'disci_layer':10}
results={}

dataset_names={'cifar100':"CIFAR100",'cards':"/kaggle/input/cards-image-datasetclassification/train",'mammals':"/kaggle/input/mammals-image-classification-dataset-45-animals/mammals"};
for c_file in conflicts_files:
    
    temp_df=pd.read_csv(os.path.join(conflict_folder,c_file),header=None)
    np_conflict=temp_df.to_numpy()
  
    counters=0;
 
    
    dist_mat= np.abs(np_conflict)
    # allclusters=example_clustering2(np_conflict,k,flag)
    key_f=c_file.split('_')[-1].split('.')[0];
    key_s=c_file.split('_')[1];
    if key_s=='layer' and key_f=='dis':
        key_f=key_f+'_layer'
    if key_f in ['ci', 'dis', 'dis_layer', 'sim']:
        print("skipped ---------------------------------------")
        print(key_f)
        continue;
    t_val=hyperParam[key_f]
    
    cluster_levels=compute_hierarchy_scipy(dist_mat,symm=True,t=t_val,criterion='distance')
    unique_clusters=[];
    for x in cluster_levels:
        if x not in unique_clusters:
            unique_clusters.append(x);
    # print(unique_clusters)
    unique_clusters=unique_clusters
    # print(unique_clusters)
    unique_clusters.reverse()
    unique_clusters=unique_clusters[1:]
    # print(key_f);
    # print(len(unique_clusters))
    # print('-------')
    import copy
    enc_sizes=[32,64,64]
    num_classes_lim=100;
    counter_1=0;
    # model=best_model
    dataset_name=dataset_names[c_file.split('_')[0]]
    if dataset_name=='CIFAR100':
        train_loader, val_loader,test_loader, num_classes, classnames= load_cifar100()
    else:
        train_loader, val_loader, test_loader,num_classes, classnames= load_folder_dataset(data_path=dataset_name)
    for level in range(len(unique_clusters)):
        clusters=unique_clusters[level]
        original_to_new_label = {}
        new_label_to_cluster = {}
        # print('training for', clusters)
        for new_label, label_group in enumerate(clusters):
           
            for original_label in label_group:
                original_to_new_label[original_label] = new_label
            new_label_to_cluster[new_label] = label_group
        num_classes_current = len(clusters)
        # print(num_classes_current)
        train_labels = np.asarray([original_to_new_label[l[1]] for l in train_loader.dataset])
        updated_train_dataset=update_data_labels(train_loader.dataset,train_labels, num_classes_current)
        val_labels = np.asarray([original_to_new_label[l[1]] for l in val_loader.dataset])
        updated_val_dataset=update_data_labels(val_loader.dataset,val_labels, num_classes_current)
        temp_train_dataloader=DataLoader(updated_train_dataset,batch_size=64,shuffle=True)
        temp_val_dataloader=DataLoader(updated_val_dataset,batch_size=64,shuffle=True)
        
        if level>0:
            # print('no copy')
            model_prev=copy.deepcopy(model)
            model=CNN2(3,enc_sizes,num_classes=num_classes_current);
            model.encoder=model_prev.encoder;
        else:
            model=CNN2(3,enc_sizes,num_classes=num_classes_current);
        
        best_model=train_model_extra_metric(model,temp_train_dataloader,temp_val_dataloader,num_epochs=5,device=device)
        counter_1+=1;
    test_labels = np.asarray([original_to_new_label[l[1]] for l in test_loader.dataset])
    updated_test_dataset=update_data_labels(test_loader.dataset,test_labels, num_classes_current)
    # test_labels = np.asarray([original_to_new_label[l[1]] for l in test_loader.dataset])
    temp_test_dataloader=DataLoader(updated_test_dataset,batch_size=64,shuffle=True)
    test_res=test_model(best_model,temp_test_dataloader)
        # if num_classes_current == num_classes_lim:
        #     break;
    results[c_file]=test_res;


   

skipped ---------------------------------------
ci
skipped ---------------------------------------
ci
skipped ---------------------------------------
dis
Epoch 1/5
Train Loss: 1.2460
Val Accuracy: 0.4477
Precision: 0.5169, Recall: 0.4477, F1: 0.4085
--------------------------------------------------
Epoch 2/5
Train Loss: 1.1918
Val Accuracy: 0.4651
Precision: 0.4882, Recall: 0.4651, F1: 0.4400
--------------------------------------------------
Epoch 3/5
Train Loss: 1.1627
Val Accuracy: 0.4869
Precision: 0.5035, Recall: 0.4869, F1: 0.4650
--------------------------------------------------
Epoch 4/5
Train Loss: 1.1315
Val Accuracy: 0.5087
Precision: 0.5597, Recall: 0.5087, F1: 0.4824
--------------------------------------------------
Epoch 5/5
Train Loss: 1.0979
Val Accuracy: 0.5000
Precision: 0.5086, Recall: 0.5000, F1: 0.4745
--------------------------------------------------
Epoch 1/5
Train Loss: 3.0743
Val Accuracy: 0.2471
Precision: 0.2834, Recall: 0.2471, F1: 0.2347
---------------

100%|██████████| 169001437/169001437 [00:11<00:00, 14486899.00it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/5
Train Loss: 0.6104
Val Accuracy: 0.8889
Precision: 0.8366, Recall: 0.8889, F1: 0.8463
--------------------------------------------------
Epoch 2/5
Train Loss: 0.4737
Val Accuracy: 0.8921
Precision: 0.8745, Recall: 0.8921, F1: 0.8526
--------------------------------------------------
Epoch 3/5
Train Loss: 0.4053
Val Accuracy: 0.9000
Precision: 0.8771, Recall: 0.9000, F1: 0.8712
--------------------------------------------------
Epoch 4/5
Train Loss: 0.3614
Val Accuracy: 0.9001
Precision: 0.8809, Recall: 0.9001, F1: 0.8731
--------------------------------------------------
Epoch 5/5
Train Loss: 0.3307
Val Accuracy: 0.9049
Precision: 0.8871, Recall: 0.9049, F1: 0.8861
--------------------------------------------------
Epoch 1/5
Train Loss: 1.8167
Val Accuracy: 0.5829
Precision: 0.5803, Recall: 0.5829, F1: 0.5280
--------------------------------------------------
Epoch 2/5
Train Loss: 1.4145

In [21]:
print(results)

{'mammals_layer_wise_disci.csv': 0.311046511627907, 'cards_layer_wise_disci.csv': 0.5418848167539267, 'cifar100_layer_wise_diseuc.csv': 0.4363, 'mammals_full_diseuc.csv': 0.30523255813953487, 'mammals_full_disci.csv': 0.3386627906976744, 'cifar100_layer_wise_disci.csv': 0.4352, 'cards_full_disci.csv': 0.5575916230366492, 'mammals_layer_wise_diseuc.csv': 0.30377906976744184, 'cards_full_diseuc.csv': 0.5575916230366492, 'cards_layer_wise_diseuc.csv': 0.5549738219895288}


In [22]:
# # dist_mat=1-conf_mat ## uncomment when using confusion matrix
# # dist_mat= np.array(conflict2d)
# dist_mat= np_conflict
# # print(dist_mat)
# cluster_levels=compute_hierarchy_scipy(dist_mat,symm=True,t=7,criterion='distance')
# # cluster_levels=compute_hierarchy_scipy(dist_mat,symm=True,method='complete',t=6,criterion='inconsistent')


In [23]:
# unique_clusters=[];
# for x in cluster_levels:
#     # if x not in unique_clusters:
#         unique_clusters.append(x);

In [24]:
# # print(unique_clusters)
# unique_clusters=unique_clusters
# # print(unique_clusters)
# unique_clusters.reverse()

In [25]:
# unique_clusters=unique_clusters[1:]
# # new_unique_clusters=[];

# # new_unique_clusters.append(unique_clusters[-1])
# # new_unique_clusters.extend(unique_clusters)
# # unique_clusters=new_unique_clusters
# # print(len(unique_clusters))
# # for x in unique_clusters:
# #     print(len(x))

In [26]:
# for x in unique_clusters:
#     print(len(x))

In [27]:
# result = []
# for sublist in unique_clusters[5]:
#     for element in sublist:
#         result.append([element])

In [28]:
# unique_clusters.append(result)

In [29]:
# for x in unique_clusters:
#     for y in x:
#         print("cluster:---------------------------------------" )
#         for z in y:
#             print(class_names[z]);
        
#     print("-------------------------------------level------------------------------------------")

# Train Function

In [30]:
# import copy
# enc_sizes=[32,64,64]
# num_classes_lim=100;
# counter_1=0;
# # model=best_model
# for level in range(len(unique_clusters)):
#     clusters=unique_clusters[level]
#     original_to_new_label = {}
#     new_label_to_cluster = {}
#     # print('training for', clusters)
#     for new_label, label_group in enumerate(clusters):
       
#         for original_label in label_group:
#             original_to_new_label[original_label] = new_label
#         new_label_to_cluster[new_label] = label_group
#     num_classes_current = len(clusters)
#     # print(num_classes_current)
#     train_labels = np.asarray([original_to_new_label[l[1]] for l in train_loader.dataset])
#     updated_train_dataset=update_data_labels(train_loader.dataset,train_labels, num_classes_current)
#     val_labels = np.asarray([original_to_new_label[l[1]] for l in val_loader.dataset])
#     updated_val_dataset=update_data_labels(val_loader.dataset,val_labels, num_classes_current)
#     temp_train_dataloader=DataLoader(updated_train_dataset,batch_size=64,shuffle=True)
#     temp_val_dataloader=DataLoader(updated_val_dataset,batch_size=64,shuffle=True)
    
#     if level>0:
#         # print('no copy')
#         model_prev=copy.deepcopy(model)
#         model=CNN2(3,enc_sizes,num_classes=num_classes_current);
#         model.encoder=model_prev.encoder;
#     else:
#         model=CNN2(3,enc_sizes,num_classes=num_classes_current);
    
#     best_model=train_model_extra_metric(model,temp_train_dataloader,temp_val_dataloader,num_epochs=3,device=device)
#     counter_1+=1;
#     # if num_classes_current == num_classes_lim:
#     #     break;

In [31]:
# test_labels = np.asarray([original_to_new_label[l[1]] for l in test_loader.dataset])
# updated_test_dataset=update_data_labels(test_loader.dataset,test_labels, num_classes_current)

In [32]:
# # test_labels = np.asarray([original_to_new_label[l[1]] for l in test_loader.dataset])
# temp_test_dataloader=DataLoader(updated_test_dataset,batch_size=64,shuffle=True)


In [33]:
# test_model(best_model,temp_test_dataloader)

In [34]:
# import copy
# enc_sizes=[32,64,128]
# num_classes_lim=100;
# counter_1=0;

# for level in range(len(unique_clusters)):
#     clusters=unique_clusters[level]
#     original_to_new_label = {}
#     new_label_to_cluster = {}
#     # print('training for', clusters)
#     for new_label, label_group in enumerate(clusters):
       
#         for original_label in label_group:
#             original_to_new_label[original_label] = new_label
#         new_label_to_cluster[new_label] = label_group
#     num_classes_current = len(clusters)
#     # print(num_classes_current)
#     train_labels = np.asarray([original_to_new_label[l[1]] for l in train_loader.dataset])
#     updated_train_dataset=update_data_labels(train_loader.dataset,train_labels, num_classes_current)
#     val_labels = np.asarray([original_to_new_label[l[1]] for l in val_loader.dataset])
#     updated_val_dataset=update_data_labels(val_loader.dataset,val_labels, num_classes_current)
#     temp_train_dataloader=DataLoader(updated_train_dataset,batch_size=64,shuffle=True)
#     temp_val_dataloader=DataLoader(updated_val_dataset,batch_size=64,shuffle=True)
    
#     if level>0:
#         model_prev=copy.deepcopy(model)
#         model=CNN(3,enc_sizes,num_classes=num_classes_current);
#         model.encoder=model_prev.encoder;
#     else:
#         model=CNN(3,enc_sizes,num_classes=num_classes_current);
    
#     best_model=train_model(model,temp_train_dataloader,temp_val_dataloader,num_epochs=5,device=device)
#     counter_1+=1;
#     if num_classes_current == num_classes_lim:
#         break;

In [35]:
best_model

CNN2(
  (encoder): Encoder(
    (encoced_blocks): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (2): ReLU()
      )
      (2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (2): ReLU()
      )
    )
  )
  (predictor): Predictor2(
    (predictor): Sequential(
      (0): Linear(in_features=1024, out_features=53, bias=True)
    )
  )
)

In [36]:
# import copy
# enc_sizes=[32,64,128]
# num_classes_lim=10;
# counter_1=0;
# for level in range(len(unique_clusters)):
#     clusters=unique_clusters[level]
#     original_to_new_label = {}
#     new_label_to_cluster = {}
#     print('training for', clusters)
#     for new_label, label_group in enumerate(clusters):
       
#         for original_label in label_group:
#             original_to_new_label[original_label] = new_label
#         new_label_to_cluster[new_label] = label_group
#     num_classes_current = len(clusters)
#     # print(num_classes_current)
#     train_labels = np.asarray([original_to_new_label[l[1]] for l in train_loader.dataset])
#     updated_train_dataset=update_data_labels(train_loader.dataset,train_labels, num_classes_current)
#     val_labels = np.asarray([original_to_new_label[l[1]] for l in val_loader.dataset])
#     updated_val_dataset=update_data_labels(val_loader.dataset,val_labels, num_classes_current)
#     temp_train_dataloader=DataLoader(updated_train_dataset,batch_size=64,shuffle=True)
#     temp_val_dataloader=DataLoader(updated_val_dataset,batch_size=64,shuffle=True)
    
#     if level>0:
#         model_prev=copy.deepcopy(model)
#         model=CNN(3,enc_sizes,num_classes=num_classes_current);
#         # model.encoder=model_prev.encoder;
#     else:
#         model=CNN(3,enc_sizes,num_classes=num_classes_current);

#     train_model(model,temp_train_dataloader,temp_val_dataloader,num_epochs=10,device=device)
#     counter_1+=1;
#     if num_classes_current == num_classes_lim:
#         break;

# Baseline Train

In [37]:
# enc_sizes=[32,64,64]
# baseline_model=CNN2(3,enc_sizes,num_classes=100)

In [38]:
# best_baseline_model=train_model_extra_metric(baseline_model,train_loader,val_loader,num_epochs=50,device=device)

In [39]:
# test_model(baseline_model,test_loader)